In [1]:
import traffic
from traffic.data import opensky
from traffic.core import Traffic
from rich.pretty import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
def download_month(month: int, year: int):

    if month < 10:
        str_month = "0" + str(month)
    else:
        str_month = str(month)


    next_month = month + 1 if month < 12 else  1
    str_next_month = str(next_month) if next_month > 9 else "0" + str(next_month)

    datetime_str_start = str(year) + "-"+ str_month + "-01 00:00:00"
    datetime_str_end = str(year) + "-" + str_next_month + "-01 00:00:00"
    datetime_current = datetime.strptime(datetime_str_start, '%Y-%m-%d %H:%M:%S')
    datetime_end = datetime.strptime(datetime_str_end, '%Y-%m-%d %H:%M:%S')

    while datetime_current < datetime_end:

        print(datetime_current)
        day = datetime_current.day
        if day < 10:
            day_str = "0" + str(day)
        else:
            day_str = str(day)

        h5_key = "LH_2205"+day_str
        str_current_day = str(datetime_current)
        str_next_day = str(datetime_current + timedelta(days = 1))
        print("getting data between "+str_current_day+ " and "+ str_next_day)

        trajectories = opensky.history(str_current_day,
                            stop= str_next_day,
                            arrival_airport ="EDDF",
                            cached=False )
        try:
            callsigns_lh = trajectories.data.callsign.loc[trajectories.data.callsign.str.contains("DLH").astype(bool)]

            callsigns_lh = callsigns_lh.unique()
            callsigns_lh = callsigns_lh[~pd.isnull(callsigns_lh)]
            trajectories_lh = trajectories[callsigns_lh]
            trajectories_lh = trajectories_lh.assign_id().eval()
            print("add",h5_key,"to file")
            trajectories_lh.to_hdf("Frankfurt_LH_2205.h5", key=h5_key, format = 'table')

        except AttributeError as e:
            print(e)

        datetime_current += timedelta(days = 1)


In [4]:
download_month(5,2022)

Output()

2022-05-01 00:00:00
getting data between 2022-05-01 00:00:00 and 2022-05-02 00:00:00


KeyboardInterrupt: 

In [6]:
# check keys of the h5-file
import h5py
filename = "Frankfurt_LH_2205.h5"
with h5py.File(filename, "r") as f:
    print(f.keys())

<KeysViewHDF5 ['LH_220301', 'LH_220302', 'LH_220303', 'LH_220304', 'LH_220305', 'LH_220306', 'LH_220307', 'LH_220308', 'LH_220309', 'LH_220310', 'LH_220311', 'LH_220312', 'LH_220313', 'LH_220314', 'LH_220315', 'LH_220316', 'LH_220317', 'LH_220318', 'LH_220319', 'LH_220320', 'LH_220321', 'LH_220322', 'LH_220323', 'LH_220324', 'LH_220325', 'LH_220326', 'LH_220327', 'LH_220328', 'LH_220329', 'LH_220330']>


In [3]:
int("05")

5